In [1]:
import numpy as np
import pandas as pd

from functools import reduce
from jinja2 import Template
import math
from StyleFrame import StyleFrame, Styler, utils
from scipy import stats
import sys
import openpyxl
import datetime
import os

sys.path.append('C:/Users/wangj/Documents/GitHub/genie')

from utils3.data_io_utils import get_df_from_pg
from utils3.data_io_utils import upload_df_to_pg
from utils3.metrics import BinWoe
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt

C:\Users\wangj\Anaconda3\lib\site-packages\StyleFrame\style_frame.py:11: DeprecationWarning:Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
C:/Users/wangj/Documents/GitHub/genie\utils3\data_io_utils.py:101: DeprecationWarning:This method will be removed in future versions.  Use 'parser.read_file()' instead.
C:\Users\wangj\Anaconda3\lib\site-packages\patsy\constraint.py:13: DeprecationWarning:Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working


In [27]:
yansheng1_sql="""
with sample as 
(SELECT id :: text as loan_id
            , apply_time
            , effective_date
            , customer_id :: text
        FROM rt_t_gocash_core_loan
        WHERE effective_date between '2020-03-04' and '2020-05-08' and return_flag = 'true' and product_id = 1
        UNION 
        SELECT loan_id
            , apply_time
            , effective_date
            , customer_id :: text                
        FROM temp_uku_oldmodelv3_sample),
sample_record as(
select distinct 
    a.customer_id,
    a.loan_id,
	d.id,
	d.contact_status,
	d.case_id,
	d.answer_status,
	d.communication_status
from
(SELECT id :: text as loan_id
            , apply_time
            , effective_date
            , customer_id :: text
        FROM rt_t_gocash_core_loan
        WHERE effective_date between '2020-03-04' and '2020-05-08' and return_flag = 'true' and product_id = 1
        UNION 
        SELECT loan_id
            , apply_time
            , effective_date
            , customer_id :: text                
        FROM temp_uku_oldmodelv3_sample)a 
left join 
(select b.customer_id,
    b.order_id as loan_id,
	b.id,
	b.contact_status,
	b.case_id,
	b.answer_status,
	b.communication_status,
	c.apply_time
from public.dw_gocash_gocash_collection_col_record b
left join public.dw_gocash_go_cash_loan_gocash_core_loan c
on b.order_id=c.id)d
on a.customer_id=d.customer_id::varchar
and a.apply_time::timestamp>d.apply_time::timestamp
)
select distinct a.*,
    call_all_num,call_self_num,call_other_num,rucui_num,call_all_success_num,call_other_success_num,all_valid_cummunication_num,self_valid_cummunication_num,
	other_valid_cummunication_num,all_promise_refund_num,self_promise_refund_num,other_promise_refund_num,all_reject_refund_num,self_reject_refund_num,other_reject_refund_num,
	all_economics_difficulty_num,self_economics_difficulty_num,other_economics_difficulty_num,all_need_to_negotiate_num,self_need_to_negotiate_num,other_need_to_negotiate_num,
	all_non_effect_communication_num,self_non_effect_communication_num,other_non_effect_communication_num
from sample a
left join 
(select customer_id,loan_id,count(distinct id) as call_all_num,
		count(distinct case when contact_status='SELF' then id else null end) as call_self_num,
		count(distinct case when contact_status<>'SELF' then id else null end) as call_other_num,
		count(distinct case_id) as rucui_num,
		count(distinct case when answer_status='SELF_ANSWER' then id else null end) as call_all_success_num,
		count(distinct case when answer_status='SELF_ANSWER' and contact_status='SELF' then id else null end) as call_self_success_num,
		count(distinct case when answer_status='SELF_ANSWER' and contact_status<>'SELF' then id else null end) as call_other_success_num,
		count(distinct case when communication_status is not null then id else null end) as all_valid_cummunication_num,
		count(distinct case when communication_status is not null and contact_status='SELF' then id else null end) as self_valid_cummunication_num,
		count(distinct case when communication_status is not null and contact_status<>'SELF' then id else null end) as other_valid_cummunication_num,
		count(distinct case when communication_status='PROMISE_REFUND' then id else null end) as all_promise_refund_num,
		count(distinct case when communication_status='PROMISE_REFUND' and contact_status='SELF' then id else null end) as self_promise_refund_num,
		count(distinct case when communication_status='PROMISE_REFUND' and contact_status<>'SELF' then id else null end) as other_promise_refund_num,
		count(distinct case when communication_status='REJECT_REFUND' then id else null end) as all_reject_refund_num,
		count(distinct case when communication_status='REJECT_REFUND' and contact_status='SELF' then id else null end) as self_reject_refund_num,
		count(distinct case when communication_status='REJECT_REFUND' and contact_status<>'SELF' then id else null end) as other_reject_refund_num,
		count(distinct case when communication_status='ECONOMICS_DIFFICULTY' then id else null end) as all_economics_difficulty_num,
		count(distinct case when communication_status='ECONOMICS_DIFFICULTY' and contact_status='SELF' then id else null end) as self_economics_difficulty_num,
		count(distinct case when communication_status='ECONOMICS_DIFFICULTY' and contact_status<>'SELF' then id else null end) as other_economics_difficulty_num,
		count(distinct case when communication_status='NEED_TO_NEGOTIATE' then id else null end) as all_need_to_negotiate_num,
		count(distinct case when communication_status='NEED_TO_NEGOTIATE' and contact_status='SELF' then id else null end) as self_need_to_negotiate_num,
		count(distinct case when communication_status='NEED_TO_NEGOTIATE' and contact_status<>'SELF' then id else null end) as other_need_to_negotiate_num,
		count(distinct case when communication_status='NON_EFFECT_COMMUNICATION' then id else null end) as all_non_effect_communication_num,
		count(distinct case when communication_status='NON_EFFECT_COMMUNICATION' and contact_status='SELF' then id else null end) as self_non_effect_communication_num,
		count(distinct case when communication_status='NON_EFFECT_COMMUNICATION' and contact_status<>'SELF' then id else null end) as other_non_effect_communication_num
		from sample_record
		group by customer_id,loan_id)b
on a.customer_id=b.customer_id::varchar
and a.loan_id=b.loan_id
"""
yansheng1_data=get_df_from_pg(yansheng1_sql)

current Function [get_df_from_pg] run time is 91.94


In [14]:
import pickle
import pandas

In [15]:
pickle.dump(yansheng_data, open('v3_yansheng_bianliang.pkl', "wb"))

In [ ]:
with open('v3_yansheng_bianliang.pkl', 'rb') as f:
    raw_data_pkl = pickle.load(f)

In [9]:
yansheng_data.drop_duplicates(inplace=True)

In [28]:
yansheng1_data.shape

(91835, 28)

In [29]:
yansheng1_data.loan_id.nunique()

91835

In [30]:
yansheng1_data.describe()

,call_all_num,call_self_num,call_other_num,rucui_num,call_all_success_num,call_other_success_num,all_valid_cummunication_num,self_valid_cummunication_num,other_valid_cummunication_num,all_promise_refund_num,...,other_reject_refund_num,all_economics_difficulty_num,self_economics_difficulty_num,other_economics_difficulty_num,all_need_to_negotiate_num,self_need_to_negotiate_num,other_need_to_negotiate_num,all_non_effect_communication_num,self_non_effect_communication_num,other_non_effect_communication_num
count,13755.000000,13755.000000,13755.000000,13755.000000,13755.000000,13755.000000,13755.000000,13755.000000,13755.000000,13755.000000,...,13755.000000,13755.000000,13755.000000,13755.000000,13755.000000,13755.000000,13755.000000,13755.000000,13755.000000,13755.000000
mean,33.784878,16.264922,17.519956,1.965467,13.965540,7.257361,33.784878,16.264922,17.519956,0.224791,...,0.001381,0.235769,0.064122,0.171647,13.350563,6.375863,6.974700,0.201091,0.056489,0.144602
std,29.728942,13.076893,19.709227,1.623466,13.765469,9.357151,29.728942,13.076893,19.709227,0.712124,...,0.039050,0.807321,0.348186,0.686485,13.515117,7.017152,9.165569,0.744022,0.306147,0.631177
min,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11.000000,7.000000,2.000000,1.000000,4.000000,0.000000,11.000000,7.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000
50%,28.000000,14.000000,12.000000,1.000000,10.000000,5.000000,28.000000,14.000000,12.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,9.000000,4.000000,4.000000,0.000000,0.000000,0.000000
75%,48.000000,22.000000,27.000000,2.000000,20.000000,11.000000,48.000000,22.000000,27.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,19.000000,8.000000,10.000000,0.000000,0.000000,0.000000
max,363.000000,167.000000,285.000000,24.000000,143.000000,110.000000,363.000000,167.000000,285.000000,17.000000,...,2.000000,18.000000,8.000000,18.000000,143.000000,132.000000,110.000000,23.000000,8.000000,23.000000


In [32]:
yansheng1_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91835 entries, 0 to 91834
Data columns (total 28 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   loan_id                             91835 non-null  object
 1   apply_time                          91835 non-null  object
 2   effective_date                      91835 non-null  object
 3   customer_id                         91835 non-null  object
 4   call_all_num                        91835 non-null  int64 
 5   call_self_num                       91835 non-null  int64 
 6   call_other_num                      91835 non-null  int64 
 7   rucui_num                           91835 non-null  int64 
 8   call_all_success_num                91835 non-null  int64 
 9   call_other_success_num              91835 non-null  int64 
 10  all_valid_cummunication_num         91835 non-null  int64 
 11  self_valid_cummunication_num        91835 non-null  in

In [33]:
yansheng1_data.head(20)

,loan_id,apply_time,effective_date,customer_id,call_all_num,call_self_num,call_other_num,rucui_num,call_all_success_num,call_other_success_num,...,other_reject_refund_num,all_economics_difficulty_num,self_economics_difficulty_num,other_economics_difficulty_num,all_need_to_negotiate_num,self_need_to_negotiate_num,other_need_to_negotiate_num,all_non_effect_communication_num,self_non_effect_communication_num,other_non_effect_communication_num
0,427266794313908224,2020-01-23 23:45:42,2020-01-24,362655285378060288,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,427271649065598976,2020-01-24 00:04:59,2020-01-24,398484911505711104,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,427278086584238080,2020-01-24 00:30:34,2020-01-24,367712255185293312,2,2,0,1,2,0,...,0,1,1,0,1,1,0,0,0,0
3,427280043176534016,2020-01-24 00:38:21,2020-01-24,388643610216202240,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,427287761790795776,2020-01-24 01:09:01,2020-01-24,417272357366169600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,427291685561991168,2020-01-24 01:24:36,2020-01-24,418421103391244288,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,427292561617887232,2020-01-24 01:28:05,2020-01-24,406717908323311616,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,427297791059525632,2020-01-24 01:48:52,2020-01-24,362638022021255168,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,427303288038850560,2020-01-24 02:10:43,2020-01-24,330463312022118400,7,7,0,2,3,0,...,0,0,0,0,3,3,0,0,0,0
9,427322516036820992,2020-01-24 03:27:07,2020-01-24,339133682430287872,4,4,0,2,3,0,...,0,0,0,0,2,2,0,0,0,0


In [37]:
pd.set_option('display.max_columns', None)
yansheng1_data.loc[yansheng1_data['customer_id']=='366520505703899136']

,loan_id,apply_time,effective_date,customer_id,call_all_num,call_self_num,call_other_num,rucui_num,call_all_success_num,call_other_success_num,all_valid_cummunication_num,self_valid_cummunication_num,other_valid_cummunication_num,all_promise_refund_num,self_promise_refund_num,other_promise_refund_num,all_reject_refund_num,self_reject_refund_num,other_reject_refund_num,all_economics_difficulty_num,self_economics_difficulty_num,other_economics_difficulty_num,all_need_to_negotiate_num,self_need_to_negotiate_num,other_need_to_negotiate_num,all_non_effect_communication_num,self_non_effect_communication_num,other_non_effect_communication_num
16,427348099051454464,2020-01-24 05:08:47,2020-01-24,366520505703899136,28,12,16,2,2,0,28,12,16,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
import pickle
pickle.dump(yansheng1_data, open('yanshengbianliangbyjinwei_data.pkl', "wb"))